In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

def get_title(soup):
    try:
        title = soup.find("span", id="productTitle").get_text(strip=True)
        return title
    except AttributeError:
        return "No Title"

def get_price(soup):
    try:
        price = soup.find("span", class_="a-price-whole").get_text(strip=True)
        return price
    except AttributeError:
        return "No Price"

def get_rating(soup):
    try:
        rating = soup.find("span", class_="a-icon-alt").get_text(strip=True)
        return rating
    except AttributeError:
        return "No Rating"

def get_review_count(soup):
    try:
        reviews = soup.find("span", id="acrCustomerReviewText").get_text(strip=True)
        return reviews
    except AttributeError:
        return "No Reviews"

def get_availability(soup):
    try:
        available = soup.find("div", id="availability").get_text(strip=True)
        return available
    except AttributeError:
        return "Not Available"

if __name__ == '__main__':
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0', 'Accept-Language':'en-US, en;q=0.5'})
    URL = 'https://www.amazon.com/s?k=playstation+4&crid=1UGS52GN968BC&sprefix=Pl%2Caps%2C565&ref=nb_sb_ss_ts-doa-p_3_2'
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, 'html.parser')
    links = soup.find_all("a", attrs={'class':'a-link-normal s-line-clamp-2 s-link-style a-text-normal'})
    links_list = []

    for link in links:
        links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[], "availability":[]}

    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)
